<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 2: Introduction to SimPy

Summer Semester 24


Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Shared resources
* Entities
* Entities Arrival / Generator

Credits: The following content is adapted from the official [carwash example](https://simpy.readthedocs.io/en/latest/examples/carwash.html) 

## Car wash example
- The Carwash example is a simulation of a carwash with a limited number of machines and a number of cars that arrive at the carwash to get cleaned.

- The carwash uses a Resource to model the limited number of washing machines. It also defines a process for washing a car.

- When a car arrives at the carwash, it requests a machine. Once it got one, it starts the carwash’s wash processes and waits for it to finish. It finally releases the machine and leaves.

- The cars are generated by another process. After creating an initial amount of cars it creates new car processes after a random time interval as long as the simulation continues.

Now we have understood the basics, we're going to give more structure. We will use the following convention:

- Resources will be modeled by a python class
- Entities will be modeled by a python class
- Entities arrivals will be modeled by a python function (*simpy process*)

This convention is not mandatory for working with Simpy, but it helps to better understand the process flow and makes the code more readable. We will review every component in the following.

In [1]:
import random
import simpy

## Shared Resources

Shared resources are another way to model Process Interaction. They form a congestion point where processes queue up in order to use them. Resources can be used by a limited number of processes at a time (e.g. a gas station can be modelled as a resource with a limited amount of fuel-pumps). Processes request these resources to become a user (or to “own” them) and have to release them once they are done.

We create the simpy resource and its capacity using the `simpy.Resource` within the `__init__` method in the class. We then define every process this resource supports as a different class method. 

In the CarWash example, washing the car is the process supported by the `CarWash` resource. Therefore, we define a `wash` method within the resource class

In [1]:
class Carwash:
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).
    """

    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        yield self.env.timeout(self.washtime)
        pct_dirt = random.randint(50, 99)
        print(f"Carwash removed {pct_dirt}% of {car}'s dirt.")

**Note the difference between the resource (`CarWash`) and the simpy resource (`self.machine = simpy.Resource()`)**

## Entities

Entities are the **things that move through the system** (e.g., products, documents, customers, phone calls, orders, raw materials, etc.)

They **arrive** to the system and **flow through it** (in most cases, using the **resources** of the system). We model the "flow" of the entity in the system as a class method. 

In the CarWash example, once a car have arrived to the system, it has to wait for a machine to become available afther which the car is washed (by the machine) and leaves the system. We define this "flow" in the `run` method within the entity class.

In [7]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}.')
        with carwash.machine.request() as request:
            yield request

            print(f'{self.name} enters the carwash at {self.env.now:.2f}.')
            yield self.env.process(carwash.wash(self.name))

            print(f'{self.name} leaves the carwash at {self.env.now:.2f}.')

The `run` method uses the `carwash` resource (and that's why is given as a parameter). 

For accessing a (simpy) resource we use the `.request()` statement. We wait for access to the resource using the `yield req` statement and finally we process (wash) the entity (car)

## Entity Generator

The **arrival / generation of entitites** is model as a process. **Note that we've separated the flow of an entity from the arrival of it**. These are two different (simpy) processes.  

In the CarWash example, a process is responsable of generating the cars. We make the entitites flow through the system (`env.process(car.run(carwash))`) within this process.

In [4]:
def car_generator(env, t_inter, carwash):
    car_count = 0

    # Create 4 initial cars
    for _ in range(4):
        car = Car(env, name=f'Car {car_count}')
        car_count += 1
        env.process(car.run(carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        car = Car(env, name=f'Car {car_count}')
        car_count += 1
        env.process(car.run(carwash))
        

## Run Simulation

Finally, we run the simulation. In general these are the steps:

1. Define "global" parameters
2. Create the simpy environment (`simpy.Environment()`)
3. Define the resources
4. Define the processes
5. Execute the simulation (`env.run`)

For steps 3, 4 and 5 we have to use the environment created in step 2

Note that the flow of the entity (`car.run` process) is called within the `car_generator` process. Therefore, we only have to call the `car_generator` "outside".

In [ ]:
# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME =  5     # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes


# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

# Exercise / Tasks

### Task 1:  Waiting and Processing Times

Let us define the:

- **Waiting time** as the time that elapses between the arrival of a car and its entry into the carwash (i.e. when it "owns" the resource).
- **Processing time** as the time that elapses between the car entering the carwash (i.e. when it "owns" the resource) and when it leaves (i.e. when it releases the resource)


**Task 1: Modify the `run` method of the `Car` class (flow of an entity) to print the waiting and processing time for each car**

**Hint:** Note that the processing time must be equal to the `WASHTIME` parameter (because we're assuming the washtime is deterministic and the same for every car)

In [14]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        # Modify this code to print the waiting and processing time for each car
        arrival_time = self.env.now
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}.')
        
        with carwash.machine.request() as request:
            
            yield request
            
            # Record the time when the car enters the carwash
            entry_time = self.env.now

            waiting_time = entry_time - arrival_time
            print(f'{self.name} enters the carwash at {entry_time:.2f}. Waiting time: {waiting_time:.2f}')

            yield self.env.process(carwash.wash(self.name))
            
            # Record the time when the car leaves the carwash
            exit_time = self.env.now
            
            # Calculate the processing time
            processing_time = exit_time - entry_time
            print(f'{self.name} leaves the carwash at {exit_time:.2f}. Processing time: {processing_time:.2f}')            

In [15]:
#Leave this code chunk as is to check your change in Car.run

# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME =  5     # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes


# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

Running Simulation...
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00. Waiting time: 0.00
Car 1 enters the carwash at 0.00. Waiting time: 0.00
Car 4 arrives at the carwash at 5.00.
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00. Processing time: 5.00
Car 1 leaves the carwash at 5.00. Processing time: 5.00
Car 2 enters the carwash at 5.00. Waiting time: 5.00
Car 3 enters the carwash at 5.00. Waiting time: 5.00
Car 5 arrives at the carwash at 10.00.
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00. Processing time: 5.00
Car 3 leaves the carwash at 10.00. Processing time: 5.00
Car 4 enters the carwash at 10.00. Waiting time: 5.00
Car 5 enters the carwash at 10.00. Waiting time: 0.00
Carwash removed 97% of Car 4's dirt.
Carwash removed 56

### Task 2: Queue Length

As we've said before, resources form a congestion point where processes queue up in order to use them. We can access to the entities that are currently in a specific `simpy.Resource()` queue by using the `.queue` attribute. 

This will return a (python) list of the entities that are in queue at that specific time. We can then check the length of the queue just by using the `len()` method for (python) lists. Check this [reference](https://www.tutorialspoint.com/python/list_len.htm) if you're new to the `len()` method.


**Task 2: Modify the `run` method of the `Car` class (flow of an entity) to print the length of the queue that each car faces when it arrives**

**Note:** Use the `Car` class from Task 1 (i.e. with the waiting and processing times)

In [47]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        # Modify this code to print the queue length that each car faces when it arrives at the carwash
        queue_lenght = len(carwash.machine.queue)
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}. Queue length: {queue_lenght}')
        with carwash.machine.request() as request:
            
            yield request

            print(f'{self.name} enters the carwash at {self.env.now:.2f}.')
            yield self.env.process(carwash.wash(self.name))

            print(f'{self.name} leaves the carwash at {self.env.now:.2f}.')     
            

In [53]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        # Modify this code to print the waiting and processing time for each car
        arrival_time = self.env.now
        
        # Modify this code to print the queue length that each car faces when it arrives at the carwash
        queue_lenght = len(carwash.machine.queue)
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}. Queue length: {queue_lenght}')
        
        with carwash.machine.request() as request:
            
            yield request
            
            # Record the time when the car enters the carwash
            entry_time = self.env.now

            waiting_time = entry_time - arrival_time
            print(f'{self.name} enters the carwash at {entry_time:.2f}. Waiting time: {waiting_time:.2f}')

            yield self.env.process(carwash.wash(self.name))
            
            # Record the time when the car leaves the carwash
            exit_time = self.env.now
            
            # Calculate the processing time
            processing_time = exit_time - entry_time
            print(f'{self.name} leaves the carwash at {exit_time:.2f}. Processing time: {processing_time:.2f}')            

In [54]:
#Leave this code chunk as is to check your change in Car.run

# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME =  5     # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes

# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

Running Simulation...
Car 0 arrives at the carwash at 0.00. Queue length: 0
Car 1 arrives at the carwash at 0.00. Queue length: 0
Car 2 arrives at the carwash at 0.00. Queue length: 0
Car 3 arrives at the carwash at 0.00. Queue length: 1
Car 0 enters the carwash at 0.00. Waiting time: 0.00
Car 1 enters the carwash at 0.00. Waiting time: 0.00
Car 4 arrives at the carwash at 5.00. Queue length: 2
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00. Processing time: 5.00
Car 1 leaves the carwash at 5.00. Processing time: 5.00
Car 2 enters the carwash at 5.00. Waiting time: 5.00
Car 3 enters the carwash at 5.00. Waiting time: 5.00
Car 5 arrives at the carwash at 10.00. Queue length: 1
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00. Processing time: 5.00
Car 3 leaves the carwash at 10.00. Processing time: 5.00
Car 4 enters the carwash at 10.00. Waiting time: 5.00
Car 5 enters 

Now that we have computed the waiting time and the queue length for every car, and assuming the following input parameters:

- `RANDOM_SEED = 42`
- `NUM_MACHINES = 2`
- `WASHTIME =  5`     
- `T_INTER = 7`       
- `SIM_TIME = 20`

Answer the following questions:

1. Which is (are) the car(s) with the longest waiting time? How long is it? 
1. Which is (are) the car(s) with the longest queue length? How long is it? 

**Answer in this markdown chunk:**

1. *Cars 2, 3 and 4 each wait the longest, waiting time is 5 minutes.*
1. *Queue length is the longest for 4-th car with the queue being 2 cars.*

### Task 3: Variable Processing Time (washtime)

The deterministic and constant washtime for every car is a little unrealistic. In general, the processing time (washtime in this example) for each entity (car) will be random (and will follow a particular probablistic distribution, we'll learn more about this in the following lectures/exercises). 

For now, we will assume that the washtime takes `WASHTIME` +/- 2

**Task 3: Modify the `wash` method of the `Carwash` class (resource) to make the washtime a random value of `WASHTIME` +/- 2**

**Hint: Use the `random.randit()` method. Check the `car_generator` process and the relationship with the `t_inter` parameter**

In [58]:
class Carwash:
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime
            
    def wash(self, car):
        # Modify this code to make the washtime a random value of WASHTIME +/- 2
        
        yield self.env.timeout(random.randint(self.washtime - 2, self.washtime + 2))
        pct_dirt = random.randint(50, 99)
        print(f"Carwash removed {pct_dirt}% of {car}'s dirt.")        

In [59]:
#Leave this code chunk as is to check your change in Carwash.wash

# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME =  5     # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes

# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

Running Simulation...
Car 0 arrives at the carwash at 0.00. Queue length: 0
Car 1 arrives at the carwash at 0.00. Queue length: 0
Car 2 arrives at the carwash at 0.00. Queue length: 0
Car 3 arrives at the carwash at 0.00. Queue length: 1
Car 0 enters the carwash at 0.00. Waiting time: 0.00
Car 1 enters the carwash at 0.00. Waiting time: 0.00
Carwash removed 65% of Car 0's dirt.
Car 0 leaves the carwash at 3.00. Processing time: 3.00
Car 2 enters the carwash at 3.00. Waiting time: 3.00
Car 4 arrives at the carwash at 5.00. Queue length: 1
Carwash removed 97% of Car 1's dirt.
Car 1 leaves the carwash at 5.00. Processing time: 5.00
Car 3 enters the carwash at 5.00. Waiting time: 5.00
Carwash removed 93% of Car 2's dirt.
Car 2 leaves the carwash at 7.00. Processing time: 4.00
Car 4 enters the carwash at 7.00. Waiting time: 2.00
Carwash removed 55% of Car 3's dirt.
Car 3 leaves the carwash at 8.00. Processing time: 3.00
Car 5 arrives at the carwash at 11.00. Queue length: 0
Car 5 enters the

How do the above questions (waiting time and queue length) change in this new scenario? **Assume the same input parameters as before.**

**Answer in this markdown chunk:**

1. *Car 3 waits for 5 min, which is the longest waiting time*
1. *Queue length is the longest for 3-rd and 4-th car, but the queue is only one car now.*

### Task 4: What if?

Now we want to "optimize" our system in such a way that no car has to wait nor faces any queue (i.e. waiting time = 0 and queue length = 0 for all cars). For that we have two options:

1. Buy additional machines in addition to the 2 we already have (`NUM_MACHINES` parameter) for a cost of 150 EUR each.
1. Upgrade our 2 current machines to be more faster, so that the washtime (`WASHTIME` parameter) is reduced. Every reduction of 1 unit of time cost 50 EUR (e.g. if `WASHTIME = 3` we have reduced the washtime by 2 and thus we have to pay 100 (2*50) EUR). Note that the washtime remains random with a value of `WASHTIME` +/- 2. Furthermore, note that the minimum possible value that `WASHTIME` can take is 2 because of the +/- 2 variability (i.e. you can not reduce more than `WASHTIME = 2`).

**Task 4: Evaluate both options and decide which is the "optimal" decision to ensure no car has to wait nor faces any queue**

**Note 1:** The options are exclusive in the sense that you can not combine them.

**Note 2:** Assume the other parameters to be the same as before (i.e. `RANDOM_SEED = 42`, `T_INTER = 7`, and `SIM_TIME = 20`)

In [75]:
#Leave this code chunk as is and only change the relevant parameters to answer the question

# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 2      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes

# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

Running Simulation...
Car 0 arrives at the carwash at 0.00. Queue length: 0
Car 1 arrives at the carwash at 0.00. Queue length: 0
Car 2 arrives at the carwash at 0.00. Queue length: 0
Car 3 arrives at the carwash at 0.00. Queue length: 1
Car 0 enters the carwash at 0.00. Waiting time: 0.00
Car 1 enters the carwash at 0.00. Waiting time: 0.00
Carwash removed 65% of Car 0's dirt.
Car 0 leaves the carwash at 0.00. Processing time: 0.00
Car 2 enters the carwash at 0.00. Waiting time: 0.00
Carwash removed 58% of Car 2's dirt.
Car 2 leaves the carwash at 1.00. Processing time: 1.00
Car 3 enters the carwash at 1.00. Waiting time: 1.00
Carwash removed 93% of Car 3's dirt.
Car 3 leaves the carwash at 1.00. Processing time: 0.00
Carwash removed 97% of Car 1's dirt.
Car 1 leaves the carwash at 2.00. Processing time: 2.00
Car 4 arrives at the carwash at 5.00. Queue length: 0
Car 4 enters the carwash at 5.00. Waiting time: 0.00
Carwash removed 87% of Car 4's dirt.
Car 4 leaves the carwash at 5.00. 

**Answer in this markdown chunk:**

*To make the queue length = 0 only by upgrading the machines is impossible as the 3-rd car will always have to wait for one of the first 2 cars to finish. Also if the washtime is 2 (the minimum possible value) the 0 car's processing time is 0.00, which is physically impossible.*
The realistic options would be:
+ To buy 2 new machines, which would cost the car wash 300 EUR
+ To buy 1 new machine and upgrade the machines by 3 time units, which would also cost the business 300EUR (would the upgrade count for 2 machines or 3?)


### Task 5: Going beyond! (Optional)

This part is optional and just for fun =) 

**It won't be taken into account in the grading process of this assingment** and its sole purpose is to challenge yourself and have fun. 

Don't worry if you can't complete it, we will cover this topics in the upcoming lectures/exercises. Try your best!

#### 5.1. Optional Task 1

What happen if we have more than one step in a system? Consider now that our system has a drying step where another machine (resource) is used to dry the cars. 

**Optional Task 1: Implement the `Dryer` class (resource) with its respective parameters and modify the `run` method of the `Car` class (flow of an entity) to include it**

In [ ]:
class Dryer:
    pass #remove this line for your implementation

In [ ]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        # Modify this code to include the drying step
        
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}.')
        with carwash.machine.request() as request:
            yield request

            print(f'{self.name} enters the carwash at {self.env.now:.2f}.')
            yield self.env.process(carwash.wash(self.name))

            print(f'{self.name} leaves the carwash at {self.env.now:.2f}.')      

#### 5.2. Optional Task 2

As you may notice, it is a bit difficult to analyze your simulation output by just printing out what is happening. In general, it would be good to have everything that is happening within the simulation environment stored somewhere and then analyze it.

**Optional Task 2: Modify the `run` method of the `Car` class (flow of an entity) to store what is hapenning in any data structure of your preference (could be a list, a dictionary, a DataFrame, etc.)**

In [ ]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        # Modify this code to store all relevant events in any data structure of your choice
        
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}.')
        with carwash.machine.request() as request:
            yield request

            print(f'{self.name} enters the carwash at {self.env.now:.2f}.')
            yield self.env.process(carwash.wash(self.name))

            print(f'{self.name} leaves the carwash at {self.env.now:.2f}.')      